# 02 — Anomaly targeting (Isolation Forest + clustering)

This notebook reproduces the full Option 1 pipeline:

1. Project to UTM and grid into 1 km cells
2. Median-aggregate geochem per cell
3. Isolation Forest anomaly scoring (baseline + noAG)
4. DBSCAN clustering into target polygons + centroid table
5. Cluster fingerprint (geochem signature)

Outputs are saved into `artifacts/`.

In [3]:
from pathlib import Path
import sys

HERE = Path.cwd()

REPO = HERE.parent

if str(REPO) not in sys.path:
    sys.path.insert(0, str(REPO))



In [4]:
import os
from pathlib import Path
import pandas as pd
import geopandas as gpd

from src.config import GridConfig, ModelConfig, ClusterConfig, FEATURES_BASELINE, FEATURES_NOAG
from src.grid import points_to_grid, aggregate_to_cells
from src.modeling import score_iforest, mark_anomalies
from src.clustering import dbscan_clusters, clusters_to_polygons, top_targets_table
from src.fingerprint import cluster_fingerprint
from src.io import write_gpkg

REPO = Path('..').resolve()
ART = REPO / 'artifacts'
ART.mkdir(exist_ok=True)


## 1) Load raw points

In [5]:
gdf = gpd.read_file(r"..\data\GEOCHEMISTRY_STREAMS_shp\GEOCHEMISTRY_STREAM_SEDIMENTS.shp")

In [6]:

if gdf.crs is None:
    gdf = gdf.set_crs('EPSG:4283')
print('rows:', len(gdf), 'CRS:', gdf.crs)


rows: 157520 CRS: EPSG:4283


## 2) Grid (UTM 53S) + aggregate to 1 km cells

In [9]:
grid_cfg = GridConfig()
pts_utm = points_to_grid(gdf, grid_cfg.utm_epsg, grid_cfg.grid_size_m)
cells = aggregate_to_cells(pts_utm, FEATURES_BASELINE)
print('cells total:', len(cells), '| mean n_points:', float(cells['n_points'].mean()))

# Save the base grid once (optional)
out_grid = ART / 'baseline' / 'grid_cells_1km.gpkg'
write_gpkg(cells, out_grid, layer='cells_1km')
# print('Saved:', out_grid)


cells total: 40293 | mean n_points: 3.9093639093639094


## 3) Isolation Forest — baseline

We filter `min_points>=5` to reduce false anomalies from sparse cells.

In [10]:
model_cfg = ModelConfig()
min_points = 5
cells_stable = cells[cells['n_points']>=min_points].copy()

scored_base, meta_base = score_iforest(cells_stable, FEATURES_BASELINE, model_cfg.contamination, model_cfg.random_state)
scored_base, thr_base = mark_anomalies(scored_base, model_cfg.contamination)
meta_base.update({'threshold': thr_base, 'min_points': min_points, 'n_anomaly_cells': int(scored_base['is_anomaly'].sum())})
meta_base


{'features_requested': ['CU_PPM',
  'PB_PPM',
  'ZN_PPM',
  'AU_PPB',
  'AS_PPM',
  'MN_PPM',
  'FE_PCT',
  'CO_PPM',
  'NI_PPM',
  'AG_PPM',
  'MO_PPM',
  'U_PPM',
  'CR_PPM',
  'BI_PPM'],
 'features_used': ['CU_PPM',
  'PB_PPM',
  'ZN_PPM',
  'AU_PPB',
  'AS_PPM',
  'MN_PPM',
  'FE_PCT',
  'CO_PPM',
  'NI_PPM',
  'AG_PPM',
  'MO_PPM',
  'U_PPM',
  'CR_PPM',
  'BI_PPM'],
 'dropped_all_nonpositive': [],
 'contamination': 0.03,
 'random_state': 42,
 'n_cells': 9030,
 'threshold': 0.5680526520188531,
 'min_points': 5,
 'n_anomaly_cells': 271}

In [12]:
out_base = ART / 'baseline' / 'ntgs_anomaly_grid_1km_stable_baseline.gpkg'
write_gpkg(scored_base, out_base, layer='iforest_min5_baseline')
# print('Saved:', out_base)


## 4) Isolation Forest — robustness (noAG)

In [13]:
scored_noag, meta_noag = score_iforest(cells_stable, FEATURES_NOAG, model_cfg.contamination, model_cfg.random_state)
scored_noag, thr_noag = mark_anomalies(scored_noag, model_cfg.contamination)
meta_noag.update({'threshold': thr_noag, 'min_points': min_points, 'n_anomaly_cells': int(scored_noag['is_anomaly'].sum())})
meta_noag


{'features_requested': ['CU_PPM',
  'PB_PPM',
  'ZN_PPM',
  'AU_PPB',
  'AS_PPM',
  'MN_PPM',
  'FE_PCT',
  'CO_PPM',
  'NI_PPM',
  'MO_PPM',
  'U_PPM',
  'CR_PPM',
  'BI_PPM'],
 'features_used': ['CU_PPM',
  'PB_PPM',
  'ZN_PPM',
  'AU_PPB',
  'AS_PPM',
  'MN_PPM',
  'FE_PCT',
  'CO_PPM',
  'NI_PPM',
  'MO_PPM',
  'U_PPM',
  'CR_PPM',
  'BI_PPM'],
 'dropped_all_nonpositive': [],
 'contamination': 0.03,
 'random_state': 42,
 'n_cells': 9030,
 'threshold': 0.5685671695673663,
 'min_points': 5,
 'n_anomaly_cells': 271}

In [15]:
out_noag = ART / 'robustness_noAG' / 'ntgs_anomaly_grid_1km_stable_noAG.gpkg'
write_gpkg(scored_noag, out_noag, layer='iforest_min5_noAG')
# print('Saved:', out_noag)


## 5) Cluster anomalies into target polygons (baseline + noAG)

In [16]:
cl_cfg = ClusterConfig()

# baseline clustering
anom_base = scored_base[scored_base['is_anomaly']==1].copy()
anom_base = dbscan_clusters(anom_base, eps_m=cl_cfg.eps_m, min_samples=cl_cfg.min_samples)
polys_base, cents_base = clusters_to_polygons(anom_base, buffer_m=cl_cfg.buffer_m)

write_gpkg(anom_base, ART/'baseline'/'ntgs_anomaly_clusters_baseline.gpkg', layer='clusters_baseline_eps2km')
write_gpkg(polys_base, ART/'baseline'/'ntgs_target_clusters_baseline.gpkg', layer='baseline_polygons')
write_gpkg(cents_base, ART/'baseline'/'ntgs_target_clusters_baseline.gpkg', layer='baseline_centroids_wgs84')

# top targets table
top_base = top_targets_table(cents_base, out_csv=str(ART/'baseline'/'top_targets_baseline.csv'))
print('Top 10 baseline targets:')
top_base.head(10)


Top 10 baseline targets:


,cluster_id,n_cells,mean_score,max_score,mean_points,lon,lat,priority_score
0,8,11,0.695475,0.713777,18.363636,137.782988,-17.174179,7.650226
1,19,10,0.629148,0.697958,8.500000,130.680362,-13.953294,6.291480
2,0,6,0.718453,0.727204,6.833333,134.171214,-23.165687,4.310718
3,11,7,0.605703,0.700771,8.142857,137.755499,-16.793047,4.239922
4,6,6,0.605058,0.662609,6.666667,137.092430,-17.947881,3.630351
5,15,6,0.603789,0.630097,9.666667,137.369051,-16.483476,3.622732
6,2,4,0.670578,0.692360,7.000000,137.503361,-18.709179,2.682310
7,13,4,0.663133,0.733584,8.250000,136.235009,-16.772232,2.652531
8,20,4,0.612178,0.636867,14.250000,130.780068,-13.564845,2.448712
9,17,4,0.607170,0.623574,7.250000,130.821675,-14.291231,2.428681


In [17]:
# noAG clustering
anom_noag = scored_noag[scored_noag['is_anomaly']==1].copy()
anom_noag = dbscan_clusters(anom_noag, eps_m=cl_cfg.eps_m, min_samples=cl_cfg.min_samples)
polys_noag, cents_noag = clusters_to_polygons(anom_noag, buffer_m=cl_cfg.buffer_m)

write_gpkg(anom_noag, ART/'robustness_noAG'/'ntgs_anomaly_clusters_noAG.gpkg', layer='clusters_noAG_eps2km')
write_gpkg(polys_noag, ART/'robustness_noAG'/'ntgs_target_clusters_noAG.gpkg', layer='noAG_polygons')
write_gpkg(cents_noag, ART/'robustness_noAG'/'ntgs_target_clusters_noAG.gpkg', layer='noAG_centroids_wgs84')

# top targets table
top_noag = top_targets_table(cents_noag, out_csv=str(ART/'robustness_noAG'/'top_targets_noAG.csv'))
print('Top 10 noAG targets:')
top_noag.head(10)


Top 10 noAG targets:


,cluster_id,n_cells,mean_score,max_score,mean_points,lon,lat,priority_score
0,8,11,0.689850,0.706780,18.363636,137.782988,-17.174179,7.588355
1,18,10,0.631166,0.698933,8.500000,130.680362,-13.953294,6.311663
2,0,6,0.725209,0.737225,6.833333,134.171214,-23.165687,4.351251
3,13,6,0.636329,0.740183,10.166667,136.230702,-16.765318,3.817972
4,6,6,0.606641,0.667617,6.666667,137.092430,-17.947881,3.639844
5,2,5,0.667977,0.725391,7.600000,137.503294,-18.709180,3.339885
6,11,5,0.616323,0.720054,8.400000,137.753859,-16.792073,3.081616
7,19,5,0.608797,0.635142,12.400000,130.781134,-13.567266,3.043983
8,15,5,0.597748,0.618885,8.400000,137.371571,-16.487341,2.988742
9,16,4,0.590683,0.618680,10.000000,137.399677,-16.461445,2.362730


## 6) Cluster fingerprint (explainability)

Fingerprint compares cluster median(log10(element)) to background median(log10(element)).

In [18]:
_, fp_base = cluster_fingerprint(scored_base, anom_base, FEATURES_BASELINE, out_csv=str(ART/'baseline'/'cluster_fingerprint_baseline.csv'))
_, fp_noag = cluster_fingerprint(scored_noag, anom_noag, FEATURES_NOAG, out_csv=str(ART/'robustness_noAG'/'cluster_fingerprint_noAG.csv'))
print('Saved fingerprint CSVs')
fp_base.head()


Saved fingerprint CSVs


,cluster_id,element,delta_log10
225,0,AG_PPM,1.176091
300,0,CR_PPM,-0.494850
75,0,AU_PPB,-0.500000
275,0,U_PPM,-1.228168
100,0,AS_PPM,-2.453318


## Done

Proceed to `03_visualize_maps.ipynb` to generate  figures.